## p-median problem

In [1]:
import numpy as np
import pandas as pd
import os
import networkx as nx
import random
import matplotlib as mp
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

from pyomo.environ import *

random.seed(42)

In [2]:
############
### Sets ###
############

X_min = -10
X_max = 10
Y_min = -10
Y_max = 10


number_customers  = 100
number_facilities = 10 

N = {k:{'x':np.round(X_min+(X_max-X_min)*random.random(),1),
        'y':np.round(Y_min+(Y_max-Y_min)*random.random(),1)}
         for k in range(1,number_customers+1)}

F = {k:{'x':np.round(X_min+(X_max-X_min)*random.random(),1),
        'y':np.round(Y_min+(Y_max-Y_min)*random.random(),1)}
         for k in range(1,number_facilities+1)}

##################
### Parameters ###
##################

D_if = {(i,f):np.sqrt((v_i['x']-v_f['x'])**2+(v_i['y']-v_f['y'])**2) 
        for i,v_i in N.items() for f,v_f in F.items()}

In [ ]:
objective_p = []
for p in range(1,number_facilities+1):

    ####################
    ### Define model ###
    ####################
    
    model = ConcreteModel()
    
    # Define sets
    model.Demand_nodes = Set(initialize=N.keys())
    model.Facilities   = Set(initialize=F.keys())
    
    # Define parameters
    model.D_if = Param(model.Demand_nodes,model.Facilities, initialize={(i,f):D_if[(i,f)] 
          for i in N.keys() for f in F.keys()}, within=NonNegativeReals)
    
    # Define decision variables
    model.y     = Var(model.Facilities, within=Binary)
    model.x     = Var(model.Demand_nodes,model.Facilities, within=Binary)
    
    # Define objective function
    model.obj = Objective(expr=sum(model.D_if[i,f]*model.x[i,f] for i in model.Demand_nodes
                                   for f in model.Facilities), sense=minimize)
    
    # Define constraints
    model.demand_node_assigned_once = ConstraintList()
    for i in model.Demand_nodes:
        model.demand_node_assigned_once.add(expr=sum(model.x[i,f] 
                                             for f in model.Facilities)==1)
        
    model.max_number_facilities = ConstraintList()
    model.max_number_facilities.add(expr=sum(model.y[f] for f in model.Facilities)<=p)
        
    model.activation_facility = ConstraintList()
    for f in model.Facilities:
        model.activation_facility.add(expr=sum(model.x[i,f] for i in model.Demand_nodes)
                                      <= len(N)*model.y[f])
    
    model.write('p-median.lp', io_options={'symbolic_solver_labels': True})
    
    # Solve the problem
    solver = SolverFactory('gurobi')
    solver.solve(model)  
    
    #%%
    
    # Print the results
    print('')
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Overall assignment cost:', np.round(model.obj(),1))
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('')
    objective_p.append(np.round(model.obj(),1))
    
    print('%%%%%%%%%%%%%%%%%%%')
    print('Used facilities:')
    print('%%%%%%%%%%%%%%%%%%%')
    used_facilities = []
    for f in model.Facilities:
        if model.y[f].value >= 0.99:
            used_facilities.append(f)
    
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Assignment of demand nodes to facilities:')
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    N_f = {k:[] for k in F.keys()}
    for i in model.Demand_nodes:
        for f in model.Facilities:
            if model.x[(i,f)].value >= 0.99:
                print(f'Node {i} - Facility {f}')
                N_f[f].append(i)
    
    axis_font  = {'fontname':'Arial', 'size':'15'}            
    plt.close('all')
    fig, ax = plt.subplots()
    
    for i,v_i in N.items():
        plt.plot(v_i['x'],v_i['y'],marker='o',markersize=5,color='k',zorder=10)
    
    for f,v_f in F.items():
        if f in used_facilities:
            plt.plot(v_f['x'],v_f['y'],marker='s',markersize=10,color='g',zorder=10)
        else:
            plt.plot(v_f['x'],v_f['y'],marker='s',markersize=10,color='r',zorder=10)
            
    
    for f in used_facilities:
        for i in N_f[f]:
            plt.plot([N[i]['x'],F[f]['x']],
                     [N[i]['y'],F[f]['y']],linewidth=1.5,color='b')
    
    ax.grid(True)
    ax.set_xlabel('X',**axis_font)
    ax.set_ylabel('Y',**axis_font)
    plt.show()
    fig.savefig('sol_pmedian_%i.png'%(p), format='png', dpi=1000, bbox_inches='tight',
             transparent=True,pad_inches=0.02) 


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Overall assignment cost: 786.3
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

%%%%%%%%%%%%%%%%%%%
Used facilities:
%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Assignment of demand nodes to facilities:
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Node 1 - Facility 7
Node 2 - Facility 7
Node 3 - Facility 7
Node 4 - Facility 7
Node 5 - Facility 7
Node 6 - Facility 7
Node 7 - Facility 7
Node 8 - Facility 7
Node 9 - Facility 7
Node 10 - Facility 7
Node 11 - Facility 7
Node 12 - Facility 7
Node 13 - Facility 7
Node 14 - Facility 7
Node 15 - Facility 7
Node 16 - Facility 7
Node 17 - Facility 7
Node 18 - Facility 7
Node 19 - Facility 7
Node 20 - Facility 7
Node 21 - Facility 7
Node 22 - Facility 7
Node 23 - Facility 7
Node 24 - Facility 7
Node 25 - Facility 7
Node 26 - Facility 7
Node 27 - Facility 7
Node 28 - Facility 7
Node 29 - Facility 7
Node 30 - Facility 7
Node 31 - Facility 7
Node 32 - Facility 7
Node 33 - Facility 7
Node 34 - Facili

In [ ]:
# Plot demand nodes and facilities
fig, ax = plt.subplots()
    
for i,v_i in N.items():
    plt.plot(v_i['x'],v_i['y'],marker='o',markersize=5,color='k',zorder=10)

for f,v_f in F.items():
    plt.plot(v_f['x'],v_f['y'],marker='s',markersize=10,color='b',zorder=10)
        
ax.grid(True)
ax.set_xlabel('X',**axis_font)
ax.set_ylabel('Y',**axis_font)
plt.show()
fig.savefig('pmedian_input.png', format='png', dpi=1000, bbox_inches='tight',
         transparent=True,pad_inches=0.02) 

# Plot objective value as a function of p
fig, ax = plt.subplots()
plt.plot(range(1,number_facilities+1),objective_p,marker='o',
         markersize=10,color='r',zorder=10)
ax.grid(True)
x_ticks_pos    = [p for p in range(1,number_facilities+1)]
x_ticks_labels = [str(p) for p in range(1,number_facilities+1)]

ax.set_xticks(x_ticks_pos,labels=x_ticks_labels)
ax.set_xlabel('p',**axis_font)
ax.set_ylabel('$\sum_{i \in \mathcal{N}}\sum_{f \in \mathcal{F}}D_{if}x_{if}$',**axis_font)
plt.show()
fig.savefig('obj_p.png', format='png', dpi=1000, bbox_inches='tight',
         transparent=True,pad_inches=0.02) 